In [ ]:
from datetime import datetime, timedelta
import os 
import pickle

import geopandas as gpd
import matplotlib.pyplot as plt
from tqdm import tqdm

import get_training_data

%load_ext autoreload
%autoreload 2

WORK_DIR = '../data/sampling_locations/'

In [ ]:
gdf = gpd.read_file(os.path.join(WORK_DIR, 'collected_locations2025-08.geojson'))
gdf.head()
    

In [ ]:
gdf.clear_threshold.value_counts()

In [ ]:
clear_threshold = 0.75  # minimum clear pixels for a tile to be included in the dataset
patch_size = 48
resolution = 10
collection = 'S2L1C'
outdir = '../data/training_patches'

training_data = get_training_data.TrainingData(patch_size=patch_size, resolution=resolution, collection=collection, 
                                               clear_threshold=clear_threshold, outdir=outdir)
    

tiles = training_data.get_patches(gdf[gdf.clear_threshold == clear_threshold])

In [ ]:
# Get some cloudy negatives too
clear_threshold = 0.1  
training_data = get_training_data.TrainingData(patch_size=patch_size, resolution=resolution, collection=collection, 
                                               clear_threshold=clear_threshold, outdir=outdir)
    

tiles = training_data.get_patches(gdf[gdf.clear_threshold == clear_threshold])